<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/LLM_JEPA_MISTRAL_FT_BTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Oct  7 02:05:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   33C    P0             61W /  400W |    5489MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
# Install necessary libraries for QLoRA and SFTTrainer
!pip install -q -U bitsandbytes transformers peft accelerate trl datasets -q

In [ ]:
!pip install pandas pandas_ta -q

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
!ls /content/gdrive/MyDrive/CryptoFT/dataset/

btc_instruction_dataset.jsonl  temp_tokenized


In [3]:
!df -h /
!du -sh /root/.cache/huggingface/

Filesystem      Size  Used Avail Use% Mounted on
overlay         236G   49G  188G  21% /
4.0K	/root/.cache/huggingface/


In [2]:
#--- CLEAN UP TEMPORARY STORAGE ---
print("\n--- Cleaning up temporary storage ---")
!rm -rf /root/.cache/huggingface/*
!rm -rf /tmp/*
print("✅ Temporary storage cleared.")


--- Cleaning up temporary storage ---
rm: cannot remove '/tmp/colab_runtime.sock': Device or resource busy
✅ Temporary storage cleared.


In [ ]:
print("--- Installing bitsandbytes from main repository ---")
!pip uninstall -y bitsandbytes
!pip install -q -U git+https://github.com/TimDettmers/bitsandbytes.git
# Verify installation
import bitsandbytes as bnb
print(f"bitsandbytes version: {bnb.__version__}")
print("✅ bitsandbytes reinstalled")

In [ ]:
# Reinstall bitsandbytes from source
print("\n--- Installing bitsandbytes from main repository ---")
!pip uninstall -y bitsandbytes
!pip install -q -U git+https://github.com/TimDettmers/bitsandbytes.git
import bitsandbytes as bnb
print(f"bitsandbytes version: {bnb.__version__}")
print("✅ bitsandbytes reinstalled")

In [ ]:
# Verify disk space
print("\n--- Checking disk space ---")
!df -h /
!du -sh /root/.cache/huggingface/ || echo "Hugging Face cache not found"

# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive/MyDrive/CryptoFT/dataset/


--- Checking disk space ---
Filesystem      Size  Used Avail Use% Mounted on
overlay         236G   42G  195G  18% /
du: cannot access '/root/.cache/huggingface/': No such file or directory
Hugging Face cache not found
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
btc_instruction_dataset.jsonl


In [ ]:
# --- 1. ENVIRONMENT SETUP AND DEPENDENCY INSTALLATION ---
print("\n--- Installing dependencies and cleaning environment ---")
!pip cache purge
!rm -rf /root/.cache/huggingface/* /root/.cache/pip/* /tmp/*
!pip uninstall -y transformers peft bitsandbytes accelerate trl datasets pandas pandas_ta huggingface_hub quanto triton numpy scipy scikit-learn torch
!pip install -q -U numpy==1.26.4
!pip install -q -U scipy==1.14.1
!pip install -q -U scikit-learn==1.5.1
!pip install -q -U torch==2.3.0
!pip install -q -U transformers==4.44.2
!pip install -q -U peft==0.12.0
!pip install -q -U bitsandbytes==0.43.3
!pip install -q -U accelerate==0.34.2
!pip install -q -U trl==0.11.1
!pip install -q -U huggingface_hub==0.25.1
!pip install -q -U quanto==0.2.0
!pip install -q -U datasets==3.0.1
!pip install -q -U triton==2.3.0
!pip install -q -U pandas pandas_ta

In [2]:
# --- FULL CODE: TRAINING + DEPLOYMENT + EVALUATION FOR MISTRAL-7B-BTC-JEPA-LLM-EXPERT ---

# --- 1. IMPORTS AND CUSTOM CLASSES ---
import os
import torch
import torch.nn.functional as F
import re
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from torch.utils.data import DataLoader
from peft import LoraConfig, get_peft_model, PeftModel
from huggingface_hub import login, HfApi, create_repo
from huggingface_hub.utils import RepositoryNotFoundError
from google.colab import drive
from tqdm import tqdm
import shutil

# Mount Google Drive
drive.mount('/content/gdrive')

# --- LLM-JEPA CONFIGURATION ---
LLM_JEPA_CONFIG = {
    "lbd": 2.0,               # Lambda (JEPA loss weight)
    "gamma": 1.0,             # Gamma (LLM loss weight)
    "predictors": 3,          # k (Number of predictor tokens)
    "last_token": -1,         # Index of last token for embedding (typically EOS/</s> or -1)
}
SPECIAL_PREDICTOR_TOKENS = [f"<|predictor_{i}|>" for i in range(1, LLM_JEPA_CONFIG["predictors"] + 1)]

# --- FILE PATHS ---
MODEL_NAME = "mistralai/Mistral-7B-v0.1"
DATASET_PATH = "/content/gdrive/MyDrive/CryptoFT/dataset/btc_instruction_dataset.jsonl"
OUTPUT_DIR = "/content/gdrive/MyDrive/CryptoFT/models/results_btc_jepa"
FINAL_MODEL_DIR = "/content/gdrive/MyDrive/CryptoFT/models/Mistral-7B-BTC-JEPA_FINAL"
HUB_MODEL_ID = "frankmorales2020/Mistral-7B-BTC-JEPA-LLM-Expert"
MAX_SEQ_LENGTH = 1024
TEMP_DATASET_DIR = "/content/gdrive/MyDrive/CryptoFT/dataset/temp_tokenized"

# --- UTILITIES ---
def get_messages_from_sample(sample_text):
    """Parses the Mistral template: '<s>[INST] Instruction [/INST] Response</s>'"""
    inst_match = re.search(r"\[INST\](.*?)\[/INST\]", sample_text, re.DOTALL)
    user_content = inst_match.group(1).strip() if inst_match else ""
    resp_match = re.search(r"\[/INST\](.*?)\s*</s>", sample_text, re.DOTALL)
    assistant_content = resp_match.group(1).strip() if resp_match else ""
    return {"user_content": user_content, "assistant_content": assistant_content}

# --- CUSTOM DATA COLLATOR ---
class CustomJEPABatchCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.keys = [
            "input_ids", "labels", "attention_mask",
            "input_ids_user", "labels_user", "attention_mask_user",
            "input_ids_assistant", "labels_assistant", "attention_mask_assistant"
        ]

    def __call__(self, features):
        batch = {key: [] for key in self.keys}
        if not features or not any(features):
            return None
        for feature in features:
            if all(key in feature for key in self.keys):
                for key in self.keys:
                    batch[key].append(feature[key])
        if not any(batch.values()):
            return None
        for key in self.keys:
            if not batch[key]:
                raise ValueError(f"No valid samples for key '{key}' in batch. Check dataset tokenization.")
            batch[key] = torch.LongTensor(batch[key])
        return batch

# --- CUSTOM TRAINER ---
class RepresentationTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        self.lbd = kwargs.pop('lbd', 1.0)
        self.gamma = kwargs.pop('gamma', 1.0)
        self.last_token = kwargs.pop('last_token', -1)
        self._last_metrics = {}  # Store metrics for logging
        super().__init__(*args, **kwargs)

    def get_train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.args.per_device_train_batch_size,
            collate_fn=self.data_collator,
            shuffle=False,
            num_workers=0
        )

    def get_eval_dataloader(self, eval_dataset=None):
        dataset = eval_dataset if eval_dataset is not None else self.eval_dataset
        return DataLoader(
            dataset,
            batch_size=self.args.per_device_eval_batch_size,
            collate_fn=self.data_collator,
            shuffle=False,
            num_workers=0
        )

    def _last_token_index(self, input_ids, attention_mask):
        last_indices = torch.sum(attention_mask, dim=1) + self.last_token
        return torch.clamp(last_indices, min=0)

    def forward(self, model, inputs):
        batch_size = inputs["input_ids"].shape[0]
        llm_inputs = {
            "input_ids": torch.cat([inputs["input_ids"], inputs["input_ids_user"], inputs["input_ids_assistant"]], dim=0),
            "labels": torch.cat([inputs["labels"], inputs["labels_user"], inputs["labels_assistant"]], dim=0),
            "attention_mask": torch.cat([inputs["attention_mask"], inputs["attention_mask_user"], inputs["attention_mask_assistant"]], dim=0),
        }
        with torch.enable_grad():
            outputs = model(**llm_inputs, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1].requires_grad_(True)
        user_hidden_states = hidden_states[batch_size: batch_size * 2]
        assistant_hidden_states = hidden_states[batch_size * 2:]
        return {
            'main_outputs': outputs,
            'user_hidden_states': user_hidden_states,
            'assistant_hidden_states': assistant_hidden_states,
        }

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        batch_size = inputs["input_ids"].shape[0]
        index_user = self._last_token_index(inputs["input_ids_user"], inputs["attention_mask_user"])
        index_assistant = self._last_token_index(inputs["input_ids_assistant"], inputs["attention_mask_assistant"])
        forward_results = self.forward(model, inputs)
        lm_loss = forward_results['main_outputs'].loss
        user_hidden_states = forward_results['user_hidden_states']
        assistant_hidden_states = forward_results['assistant_hidden_states']
        user_embedding = user_hidden_states[range(batch_size), index_user, :]
        assistant_embedding = assistant_hidden_states[range(batch_size), index_assistant, :]
        cosine_similarity = F.cosine_similarity(user_embedding, assistant_embedding, dim=-1)
        jepa_loss = 1.0 - torch.mean(cosine_similarity)
        total_loss = self.gamma * lm_loss + self.lbd * jepa_loss
        self._last_metrics = {'jepa_loss': jepa_loss, 'lm_loss': lm_loss}

        # FIXED: Return only loss tensor for training, metrics are stored separately
        if return_outputs:
            return (total_loss, forward_results['main_outputs'])
        return total_loss

    def log(self, logs):
        if self._last_metrics:
            logs["jepa_loss"] = self._nested_gather(self._last_metrics["jepa_loss"]).mean().item()
            logs["lm_loss"] = self._nested_gather(self._last_metrics["lm_loss"]).mean().item()
        super().log(logs)

# --- 2. MODEL AND DATASET SETUP ---
print("\n--- Setting up model and dataset ---")
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(FINAL_MODEL_DIR, exist_ok=True)
os.makedirs(TEMP_DATASET_DIR, exist_ok=True)

# Clear any existing dataset cache
if os.path.exists(TEMP_DATASET_DIR):
    shutil.rmtree(TEMP_DATASET_DIR)
os.makedirs(TEMP_DATASET_DIR, exist_ok=True)

# PEFT (LoRA) Config
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)


# Training Arguments - OPTIMIZED FOR JEPA MONITORING
training_arguments = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    optim="paged_adamw_8bit",
    save_steps=0,  # Disable saving during demo
    logging_steps=10,  # See JEPA metrics every 10 steps
    max_steps=50,  # Just enough to see JEPA loss decreasing
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    disable_tqdm=False,
    report_to="none",
    # ↓↓↓ CRITICAL CHANGES FOR DEMO ↓↓↓
    evaluation_strategy="no",  # Disable evaluation during training
    eval_steps=None,  # No evaluation steps
    metric_for_best_model=None,  # Not needed for demo
    # ↑↑↑ CRITICAL CHANGES FOR DEMO ↑↑↑
    dataloader_drop_last=True,
    dataloader_num_workers=0,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    resume_from_checkpoint=True
)

# Load Model with Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.add_special_tokens({"additional_special_tokens": SPECIAL_PREDICTOR_TOKENS})
model.resize_token_embeddings(len(tokenizer))
model = get_peft_model(model, peft_config)
for name, param in model.named_parameters():
    if "lora" in name:
        param.requires_grad = True
print("✅ PEFT (LoRA) configuration applied")

# Load and Prepare Dataset
print("\n--- Loading and Splitting Custom Dataset ---")
dataset = load_dataset("json", data_files=DATASET_PATH, split="train")

# Filter malformed samples
def filter_valid_samples(example):
    text = example["text"]
    parsed = get_messages_from_sample(text)
    valid = bool(parsed["user_content"] and parsed["assistant_content"])
    if not valid:
        print(f"Skipping malformed sample: {text[:50]}...")
    return valid

filtered_dataset = dataset.filter(filter_valid_samples, desc="Filtering valid samples")
if len(filtered_dataset) < 3000:
    raise ValueError(f"Filtered dataset has {len(filtered_dataset)} samples, need at least 3000 for 2500 train + 500 eval.")
print(f"Filtered dataset size: {len(filtered_dataset)} samples")

# Select 2500 for train, 500 for eval
train_dataset = filtered_dataset.select(range(2500))
test_dataset = filtered_dataset.select(range(2500, 3000))
tokenized_dataset = {"train": train_dataset, "test": test_dataset}

def tokenize_jepa_views(example):
    sample_text = example["text"]
    parsed = get_messages_from_sample(sample_text)
    predictor_str = "".join(SPECIAL_PREDICTOR_TOKENS)
    user_text_with_pred = f"<s>[INST] {parsed['user_content']} {predictor_str} [/INST]"
    assistant_text = f"{parsed['assistant_content']}</s>"
    tokenized_full = tokenizer(sample_text, truncation=True, max_length=MAX_SEQ_LENGTH, padding="max_length", return_tensors="np")
    tokenized_user = tokenizer(user_text_with_pred, truncation=True, max_length=MAX_SEQ_LENGTH, padding="max_length", return_tensors="np")
    tokenized_assistant = tokenizer(assistant_text, truncation=True, max_length=MAX_SEQ_LENGTH, padding="max_length", return_tensors="np")

    return {
        "input_ids": tokenized_full["input_ids"][0].tolist(),
        "attention_mask": tokenized_full["attention_mask"][0].tolist(),
        "labels": tokenized_full["input_ids"][0].copy().tolist(),
        "input_ids_user": tokenized_user["input_ids"][0].tolist(),
        "attention_mask_user": tokenized_user["attention_mask"][0].tolist(),
        "labels_user": [-100] * MAX_SEQ_LENGTH,
        "input_ids_assistant": tokenized_assistant["input_ids"][0].tolist(),
        "attention_mask_assistant": tokenized_assistant["attention_mask"][0].tolist(),
        "labels_assistant": [-100] * MAX_SEQ_LENGTH
    }

# Tokenize and save to disk to avoid caching issues
tokenized_dataset["train"] = tokenized_dataset["train"].map(
    tokenize_jepa_views,
    batched=False,
    remove_columns=["text"],
    desc="Tokenizing train samples for LLM-JEPA views",
    load_from_cache_file=False
)
tokenized_dataset["test"] = tokenized_dataset["test"].map(
    tokenize_jepa_views,
    batched=False,
    remove_columns=["text"],
    desc="Tokenizing test samples for LLM-JEPA views",
    load_from_cache_file=False
)
tokenized_dataset["train"].save_to_disk(os.path.join(TEMP_DATASET_DIR, "train"))
tokenized_dataset["test"].save_to_disk(os.path.join(TEMP_DATASET_DIR, "test"))

# Reload dataset to ensure consistency
from datasets import load_from_disk
tokenized_dataset = {
    "train": load_from_disk(os.path.join(TEMP_DATASET_DIR, "train")),
    "test": load_from_disk(os.path.join(TEMP_DATASET_DIR, "test"))
}


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
btc_instruction_dataset.jsonl  temp_tokenized

--- Setting up model and dataset ---


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ PEFT (LoRA) configuration applied

--- Loading and Splitting Custom Dataset ---


Filtering valid samples:   0%|          | 0/89685 [00:00<?, ? examples/s]

Filtered dataset size: 89685 samples


Tokenizing train samples for LLM-JEPA views:   0%|          | 0/2500 [00:00<?, ? examples/s]

Tokenizing test samples for LLM-JEPA views:   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

In [3]:
# Validate all samples in dataset
print("\n--- Validating Dataset Keys ---")
required_keys = [
    "input_ids", "labels", "attention_mask",
    "input_ids_user", "labels_user", "attention_mask_user",
    "input_ids_assistant", "labels_assistant", "attention_mask_assistant"
]
for split in ["train", "test"]:
    print(f"\nChecking {split} split...")
    missing_samples = []
    for idx, sample in tqdm(enumerate(tokenized_dataset[split]), total=len(tokenized_dataset[split]), desc=f"Validating {split} split"):
        missing_keys = [key for key in required_keys if key not in sample]
        if missing_keys:
            missing_samples.append((idx, missing_keys))
    if missing_samples:
        print(f"Found {len(missing_samples)} problematic samples in {split} split:")
        for idx, missing_keys in missing_samples[:5]:
            print(f"Sample {idx} missing keys: {missing_keys}")
        raise ValueError(f"Dataset validation failed in {split} split.")
    print(f"{split} split: All {len(tokenized_dataset[split])} samples validated.")

COLUMNS_TO_KEEP = required_keys
try:
    tokenized_dataset["train"] = tokenized_dataset["train"].select_columns(COLUMNS_TO_KEEP)
    tokenized_dataset["test"] = tokenized_dataset["test"].select_columns(COLUMNS_TO_KEEP)
    print(f"Dataset columns filtered to: {COLUMNS_TO_KEEP}")
except Exception as e:
    print(f"Error during column selection: {e}")
    print(f"Available columns in train: {tokenized_dataset['train'].column_names}")
    print(f"Available columns in test: {tokenized_dataset['test'].column_names}")
    raise



--- Validating Dataset Keys ---

Checking train split...


Validating train split: 100%|██████████| 2500/2500 [00:09<00:00, 257.38it/s]


train split: All 2500 samples validated.

Checking test split...


Validating test split: 100%|██████████| 500/500 [00:01<00:00, 261.20it/s]

test split: All 500 samples validated.
Dataset columns filtered to: ['input_ids', 'labels', 'attention_mask', 'input_ids_user', 'labels_user', 'attention_mask_user', 'input_ids_assistant', 'labels_assistant', 'attention_mask_assistant']


In [4]:
# Debug dataset before training
print("\n--- Debugging Dataset Before Training ---")
for split in ["train", "test"]:
    print(f"\n{split} split sample keys:")
    for idx in range(min(5, len(tokenized_dataset[split]))):
        sample = tokenized_dataset[split][idx]
        print(f"Sample {idx} keys: {list(sample.keys())}")

# --- 3. TRAINER INITIALIZATION AND EXECUTION ---
print("\n--- Initializing and Starting RepresentationTrainer (LLM-JEPA) ---")
custom_collator = CustomJEPABatchCollator(tokenizer)

# Log GPU memory before training
def log_gpu_memory():
    if torch.cuda.is_available():
        print(f"GPU Memory Allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MiB")
        print(f"GPU Memory Reserved: {torch.cuda.memory_reserved() / 1024**2:.2f} MiB")
        print(f"GPU Memory Free: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_reserved()) / 1024**2:.2f} MiB")

print("\n--- GPU Memory Before Training ---")
log_gpu_memory()

# ADD FIXED JEPA MONITORING CALLBACK HERE
from transformers import TrainerCallback

class JEPAMonitorCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        # Force JEPA metrics display every 10 steps (matching logging_steps=10)
        if state.global_step % 10 == 0 and hasattr(trainer, '_last_metrics'):
            metrics = trainer._last_metrics
            jepa_loss = metrics.get('jepa_loss')
            lm_loss = metrics.get('lm_loss')
            if jepa_loss is not None and lm_loss is not None:
                print(f"\n🎯 JEPA Metrics [Step {state.global_step}]:")
                print(f"   JEPA Loss: {jepa_loss.item():.4f}")
                print(f"   LM Loss: {lm_loss.item():.4f}")
                print(f"   Cosine Sim: {1 - jepa_loss.item():.4f}")
                print(f"   Total Loss: {trainer.gamma * lm_loss + trainer.lbd * jepa_loss:.4f}")

    def on_log(self, args, state, control, logs=None, **kwargs):
        # Also add to official logs for potential WandB/etc.
        if logs is not None and hasattr(trainer, '_last_metrics'):
            jepa_loss = trainer._last_metrics.get('jepa_loss')
            lm_loss = trainer._last_metrics.get('lm_loss')
            if jepa_loss is not None:
                logs["jepa_loss"] = jepa_loss.item()
                logs["lm_loss"] = lm_loss.item()

trainer = RepresentationTrainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    args=training_arguments,
    data_collator=custom_collator,
    lbd=LLM_JEPA_CONFIG["lbd"],
    gamma=LLM_JEPA_CONFIG["gamma"],
    last_token=LLM_JEPA_CONFIG["last_token"],
    callbacks=[JEPAMonitorCallback()]  # ← ADD THIS LINE
)

max_steps is given, it will override any value given in num_train_epochs



--- Debugging Dataset Before Training ---

train split sample keys:
Sample 0 keys: ['input_ids', 'labels', 'attention_mask', 'input_ids_user', 'labels_user', 'attention_mask_user', 'input_ids_assistant', 'labels_assistant', 'attention_mask_assistant']
Sample 1 keys: ['input_ids', 'labels', 'attention_mask', 'input_ids_user', 'labels_user', 'attention_mask_user', 'input_ids_assistant', 'labels_assistant', 'attention_mask_assistant']
Sample 2 keys: ['input_ids', 'labels', 'attention_mask', 'input_ids_user', 'labels_user', 'attention_mask_user', 'input_ids_assistant', 'labels_assistant', 'attention_mask_assistant']
Sample 3 keys: ['input_ids', 'labels', 'attention_mask', 'input_ids_user', 'labels_user', 'attention_mask_user', 'input_ids_assistant', 'labels_assistant', 'attention_mask_assistant']
Sample 4 keys: ['input_ids', 'labels', 'attention_mask', 'input_ids_user', 'labels_user', 'attention_mask_user', 'input_ids_assistant', 'labels_assistant', 'attention_mask_assistant']

test split

In [6]:

print("✅ JEPA Monitoring Enabled - will show metrics every 50 steps")
print("\n--- Starting LLM-JEPA Fine-Tuning with Monitoring ---")
trainer.train()
print("\n--- Fine-Tuning Complete! ---")

# Clean up old checkpoints
print("\n--- Cleaning Up Old Checkpoints ---")
shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Save to Google Drive
print("\n--- Saving Final Adapter Weights to Google Drive ---")
trainer.model.save_pretrained(FINAL_MODEL_DIR)
tokenizer.save_pretrained(FINAL_MODEL_DIR)
print(f"\n✅ LoRA adapter and tokenizer saved to: '{FINAL_MODEL_DIR}'")
print(f"Checkpoints and logs saved to '{OUTPUT_DIR}'")

✅ JEPA Monitoring Enabled - will show metrics every 50 steps

--- Starting LLM-JEPA Fine-Tuning with Monitoring ---


Step,Training Loss
10,1.531600
20,0.327500
30,0.273100
40,0.253400
50,0.269500



🎯 JEPA Metrics [Step 10]:
   JEPA Loss: 0.0089
   LM Loss: 0.5007
   Cosine Sim: 0.9911
   Total Loss: 0.5185

🎯 JEPA Metrics [Step 20]:
   JEPA Loss: 0.0009
   LM Loss: 0.2892
   Cosine Sim: 0.9991
   Total Loss: 0.2910

🎯 JEPA Metrics [Step 30]:
   JEPA Loss: 0.0003
   LM Loss: 0.2530
   Cosine Sim: 0.9997
   Total Loss: 0.2537

🎯 JEPA Metrics [Step 40]:
   JEPA Loss: 0.0002
   LM Loss: 0.2591
   Cosine Sim: 0.9998
   Total Loss: 0.2596

🎯 JEPA Metrics [Step 50]:
   JEPA Loss: 0.0002
   LM Loss: 0.2725
   Cosine Sim: 0.9998
   Total Loss: 0.2729


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(



--- Fine-Tuning Complete! ---

--- Cleaning Up Old Checkpoints ---

--- Saving Final Adapter Weights to Google Drive ---


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(



✅ LoRA adapter and tokenizer saved to: '/content/gdrive/MyDrive/CryptoFT/models/Mistral-7B-BTC-JEPA_FINAL'
Checkpoints and logs saved to '/content/gdrive/MyDrive/CryptoFT/models/results_btc_jepa'


In [ ]:
from google.colab import userdata

# --- 4. DEPLOYMENT TO HUGGING FACE ---
print("\n--- Authenticating with Hugging Face Hub ---")
try:
    access_token_write = userdata.get('HF_TOKEN')
    if not access_token_write:
        raise ValueError("HF_TOKEN secret not found or is empty.")
    login(token=access_token_write, add_to_git_credential=True)
    api = HfApi(token=access_token_write)
    print("✅ Successfully logged into Hugging Face Hub.")
except Exception as e:
    print(f"FATAL ERROR during login: {e}")
    raise

print(f"\n--- Checking/Creating Repository: {HUB_MODEL_ID} ---")
try:
    api.repo_info(repo_id=HUB_MODEL_ID, repo_type="model")
    print(f"✅ Repository {HUB_MODEL_ID} already exists.")
except RepositoryNotFoundError:
    print(f"⚠️ Repository {HUB_MODEL_ID} not found. Creating it now...")
    create_repo(repo_id=HUB_MODEL_ID, repo_type="model", private=False, token=access_token_write)
    print(f"✅ Repository {HUB_MODEL_ID} created successfully.")

print(f"\n--- Pushing LoRA Adapter and Tokenizer ---")
if not os.path.exists(FINAL_MODEL_DIR):
    raise FileNotFoundError(f"Adapter directory not found at {FINAL_MODEL_DIR}.")
try:
    trainer.model.push_to_hub(
        repo_id=HUB_MODEL_ID,
        commit_message="Initial QLoRA adapter for Bitcoin price prediction",
        private=False
    )
    tokenizer.push_to_hub(HUB_MODEL_ID)
    print("\n✅ SUCCESS: Adapter and tokenizer uploaded to Hugging Face Hub.")
except Exception as e:
    print(f"\n⚠️ WARNING: Trainer push failed. Using HfApi fallback. Error: {e}")
    api.upload_folder(
        folder_path=FINAL_MODEL_DIR,
        repo_id=HUB_MODEL_ID,
        commit_message="LoRA adapter and tokenizer pushed from Google Drive checkpoint",
        ignore_patterns=["*.pt", "*.bin", "optimizer.pt", "scheduler.pt", "rng_state.pth"]
    )
    print("\n✅ SUCCESS (Fallback): Adapter and tokenizer uploaded to Hugging Face Hub.")
print(f"\nDeployment Complete. Model available at: https://huggingface.co/{HUB_MODEL_ID}")


--- Authenticating with Hugging Face Hub ---
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful
✅ Successfully logged into Hugging Face Hub.

--- Checking/Creating Repository: frankmorales2020/Mistral-7B-BTC-JEPA-LLM-Expert ---
⚠️ Repository frankmorales2020/Mistral-7B-BTC-JEPA-LLM-Expert not found. Creating it now...
✅ Repository frankmorales2020/Mistral-7B-BTC-JEPA-LLM-Expert created successfully.

--- Pushing LoRA Adapter and Tokenizer ---


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

In [ ]:
# --- 5. EVALUATION/INFERENCE ---
print("\n--- Running Single Inference Test ---")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
USER_PROMPT_CONTENT = "Current BTC data: [O:30000, H:30500, C:30200]. Give ONLY the 12-hour direction (UP, DOWN, or FLAT)."

print(f"--- Loading Model onto {DEVICE} ---")
try:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )
    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
    )
    tokenizer = AutoTokenizer.from_pretrained(HUB_MODEL_ID)
    tokenizer.pad_token = tokenizer.eos_token
    model = PeftModel.from_pretrained(base_model, HUB_MODEL_ID).eval()
    print("✅ Model loaded successfully.")
except Exception as e:
    print(f"FATAL ERROR during model loading: {e}")
    raise

input_text = f"<s>[INST] {USER_PROMPT_CONTENT} [/INST]"
inputs = tokenizer(input_text, return_tensors="pt").to(DEVICE)
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=120,
        do_sample=False,
        num_beams=1
    )
response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
prompt_end_marker = "[/INST]"
response_start_index = response_text.rfind(prompt_end_marker) + len(prompt_end_marker)
prediction_output = response_text[response_start_index:].strip()

print(f"\n✅ Inference Complete.")
print("-" * 70)
print(f"INPUT PROMPT:\n{input_text.strip()}")
print("-" * 70)
print(f"MODEL RESPONSE (Full Output):\n{prediction_output}")
print("-" * 70)